In [13]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [14]:
# ============================================
# Minimal, weighted CV pipeline using sklearn trees
# - CV over (max_depth, min_leaf_weight in exposure units)
# - CV over ccp_alpha from cost_complexity_pruning_path
# - Final fit + test metrics
# ============================================

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

# ---------- weighted metrics ----------
def wmae(y, yhat, w):
    w = np.asarray(w, float)
    return (np.abs(y - yhat) * w).sum() / np.clip(w.sum(), 1e-12, None)

def wrmse(y, yhat, w):
    w = np.asarray(w, float)
    return np.sqrt(((y - yhat)**2 * w).sum() / np.clip(w.sum(), 1e-12, None))

def weighted_r2(y, yhat, w):
    w = np.asarray(w, float)
    ybar = (y * w).sum() / np.clip(w.sum(), 1e-12, None)
    ss_res = ((y - yhat)**2 * w).sum()
    ss_tot = ((y - ybar)**2 * w).sum()
    return 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0

def poisson_deviance(y_counts, exposure, yhat_rate):
    y   = np.asarray(y_counts,  float)
    exp = np.asarray(exposure,  float)
    lam = np.asarray(yhat_rate, float) * exp
    bad = (exp <= 0) & (y > 0)
    if np.any(bad):
        y, exp, lam = y[~bad], exp[~bad], lam[~bad]
    lam = np.clip(lam, 1e-12, None)
    term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
    return 2.0 * np.nansum(term)

def calibration_table(pred_rate, y_counts, exposure, n_bins=12):
    pred_rate = np.asarray(pred_rate, float)
    y_counts  = np.asarray(y_counts, float)
    exposure  = np.asarray(exposure, float)

    # ignore impossible rows
    ok = exposure > 0
    pred_rate, y_counts, exposure = pred_rate[ok], y_counts[ok], exposure[ok]

    # bin by predicted rate (exposure-weighted quantiles)
    q = np.quantile(pred_rate, np.linspace(0, 1, n_bins + 1))
    # make bins unique & robust
    q = np.unique(q)
    if q.size <= 2:
        # not enough variety; single bin fallback
        bins = np.zeros_like(pred_rate, int)
        edges = np.array([pred_rate.min(), pred_rate.max()])
    else:
        bins = np.minimum(np.digitize(pred_rate, q[1:-1], right=True), q.size - 2)
        edges = q

    df = pd.DataFrame({
        "bin": bins,
        "y": y_counts,
        "exp": exposure,
        "pred": pred_rate
    })
    agg = df.groupby("bin").agg(
        exposure=("exp","sum"),
        observed=("y","sum"),
        expected=("pred","sum")  # because pred is per-unit exposure; we’ll multiply below
    ).reset_index()

    # expected counts = sum(pred_rate * exposure)
    # we had sum(pred) above, so fix it:
    # better: recompute expected per-bin explicitly
    agg["expected"] = (
        df.groupby("bin").apply(lambda g: np.sum(g["pred"].to_numpy(float) * g["exp"].to_numpy(float)))
    ).to_numpy(float)

    agg["OE"] = agg["observed"] / np.clip(agg["expected"], 1e-12, None)
    return agg[["bin","exposure","observed","expected","OE"]]

def pearson_overdispersion(y_counts, exposure, pred_rate):
    mu = np.clip(pred_rate * exposure, 1e-12, None)
    y  = np.asarray(y_counts, float)
    chi2 = np.sum((y - mu)**2 / mu)
    df = max(len(y) - 1, 1)
    return chi2 / df

In [ ]:
# ---------- data ----------
from preprocessing.preprocessing_utils import preprocess_for_tree

train = pd.read_csv("../data/claims_train.csv")
test  = pd.read_csv("../data/claims_test.csv")

X_tr, y_tr_rate, w_tr = preprocess_for_tree(train)
X_te, y_te_rate, w_te = preprocess_for_tree(test)

# counts for deviance (optional)
y_tr_cnt = (y_tr_rate * w_tr).to_numpy(float)
y_te_cnt = (y_te_rate * w_te).to_numpy(float)

# arrays
X = X_tr.values.astype(np.float32)
y = y_tr_rate.values.astype(np.float32)
w = w_tr.values.astype(np.float32)

X_test = X_te.values.astype(np.float32)
y_test = y_te_rate.values.astype(np.float32)
w_test = w_te.values.astype(np.float32)

# ---------- hyperparams ----------
DEPTH_GRID = [12, 14]
LEAFW_GRID = [16.0, 18.0, 20.0]    # exposure units (sum of weights in a leaf >= this)
K_FOLDS    = 5
SEED       = 7
MAX_ALPHA_POINTS = 30

kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED)

# ============================================
# 1) CV over (max_depth, min_leaf_weight) with weighted WMAE
#    (convert exposure threshold -> min_weight_fraction_leaf per fold)
# ============================================
best_pre = {"max_depth": None, "min_leaf_weight": None, "mean_dev": np.inf}

for d in DEPTH_GRID:
    for mlw in LEAFW_GRID:
        fold_scores = []
        for tr_idx, va_idx in kf.split(X):
            Xtr, ytr, wtr = X[tr_idx], y[tr_idx], w[tr_idx]
            Xva, yva, wva = X[va_idx], y[va_idx], w[va_idx]

            frac = float(mlw / np.clip(wtr.sum(), 1e-12, None))
            frac = float(np.clip(frac, 0.0, 0.4999))  # sklearn requires < 0.5

            model = DecisionTreeRegressor(
                criterion="squared_error",
                splitter="best",
                max_depth=int(d),
                min_weight_fraction_leaf=frac,
                ccp_alpha=0.0,
                random_state=SEED
            )
            model.fit(Xtr, ytr, sample_weight=wtr)
            yhat = model.predict(Xva)
            yva_cnt = yva * wva  # counts on validation fold
            fold_scores.append(poisson_deviance(yva_cnt, wva, yhat))

        mean_dev = float(np.mean(fold_scores))
        print(f"[pre] depth={d}, min_leaf_w={mlw:.1f} -> CV Dev={mean_dev:.6f}")
        if mean_dev < best_pre["mean_dev"]:
            best_pre = {"max_depth": int(d), "min_leaf_weight": float(mlw), "mean_dev": mean_dev}

print("Chosen pre-pruning:", best_pre)

C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\2909726594.py:37: RuntimeWarning: divide by zero encountered in log
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\2909726594.py:37: RuntimeWarning: invalid value encountered in multiply
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\2909726594.py:37: RuntimeWarning: divide by zero encountered in log
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\2909726594.py:37: RuntimeWarning: invalid value encountered in multiply
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\2909726594.py:37: RuntimeWarning: divide by zero encountered in log
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\2909726594.py:37: RuntimeWarning: invalid v

[pre] depth=12, min_leaf_w=16.0 -> CV Dev=37737.585051


C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\2909726594.py:37: RuntimeWarning: divide by zero encountered in log
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\2909726594.py:37: RuntimeWarning: invalid value encountered in multiply
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)


KeyError: 'mean_dev'

In [ ]:
# ============================================
# 2) Build compact α-grid from CCP path (on full train with chosen pre-caps)
# ============================================
frac_full = float(best_pre["min_leaf_weight"] / np.clip(w.sum(), 1e-12, None))
frac_full = float(np.clip(frac_full, 0.0, 0.4999))

seed_tree = DecisionTreeRegressor(
    criterion="squared_error",
    splitter="best",
    max_depth=best_pre["max_depth"],
    min_weight_fraction_leaf=frac_full,
    random_state=SEED
)
seed_tree.fit(X, y, sample_weight=w)
path = seed_tree.cost_complexity_pruning_path(X, y, sample_weight=w)
alphas = np.unique(np.asarray(path.ccp_alphas, float))
if alphas.size == 0:
    alphas = np.array([0.0], float)
if alphas[0] > 0.0:
    alphas = np.insert(alphas, 0, 0.0)
if alphas.size > MAX_ALPHA_POINTS:
    qs = np.linspace(0, 1, MAX_ALPHA_POINTS)
    alphas = np.unique(np.quantile(alphas, qs))


In [ ]:
# ============================================
# 3) CV over α with same pre-pruning caps (weighted WMAE)
# ============================================
alpha_scores = np.zeros_like(alphas, dtype=float)

for tr_idx, va_idx in kf.split(X):
    Xtr, ytr, wtr = X[tr_idx], y[tr_idx], w[tr_idx]
    Xva, yva, wva = X[va_idx], y[va_idx], w[va_idx]

    frac_fold = float(best_pre["min_leaf_weight"] / np.clip(wtr.sum(), 1e-12, None))
    frac_fold = float(np.clip(frac_fold, 0.0, 0.4999))

    for i, a in enumerate(alphas):
        model = DecisionTreeRegressor(
            criterion="squared_error",
            splitter="best",
            max_depth=best_pre["max_depth"],
            min_weight_fraction_leaf=frac_fold,
            ccp_alpha=float(a),
            random_state=SEED
        )
        model.fit(Xtr, ytr, sample_weight=wtr)
        yhat = model.predict(Xva)
        yva_cnt = yva * wva
        alpha_scores[i] += poisson_deviance(yva_cnt, wva, yhat)

alpha_scores /= K_FOLDS
i_best = int(np.argmin(alpha_scores))
best_alpha = float(alphas[i_best])
print("Chosen α via CV:", {"alpha": best_alpha, "mean_dev": float(alpha_scores[i_best])})

Chosen α via CV: {'alpha': 3.0369141184607657e-07, 'mean_wmae': 0.1817476217580259}


In [ ]:
# ============================================
# 4) Final fit on ALL training + test evaluation
# ============================================
final = DecisionTreeRegressor(
    criterion="squared_error",
    splitter="best",
    max_depth=best_pre["max_depth"],
    min_weight_fraction_leaf=frac_full,
    ccp_alpha=best_alpha,
    random_state=SEED
)
final.fit(X, y, sample_weight=w)

yhat_te = final.predict(X_test)
calib = calibration_table(yhat_te, y_te_cnt, w_test, n_bins=12)
print("\nCalibration by predicted-rate bins (O/E):")
print(calib.to_string(index=False))

print("Pearson over-dispersion (test):",
      pearson_overdispersion(y_te_cnt, w_test, yhat_te))

print("\n=== TEST METRICS (final pruned) ===")
print("WMAE        :", wmae(y_test, yhat_te, w_test))
print("WRMSE       :", wrmse(y_test, yhat_te, w_test))
print("Weighted R^2:", weighted_r2(y_test, yhat_te, w_test))
print("Poisson Dev.:", poisson_deviance(y_te_cnt, w_test, yhat_te))


=== TEST METRICS (final pruned) ===
WMAE        : 0.1828435645410214
WRMSE       : 0.7744725602915187
Weighted R^2: 0.013614787368146919
Poisson Dev.: 48444.46224273751


C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\812682618.py:37: RuntimeWarning: divide by zero encountered in log
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
C:\Users\Emil\AppData\Local\Temp\ipykernel_11408\812682618.py:37: RuntimeWarning: invalid value encountered in multiply
  term = np.where(y > 0, y * np.log(y / lam), 0.0) - (y - lam)
